In [2]:
import ujson

In [11]:
filename = 'MOREANDMOREDATA.json' ## insert filename here
with open(filename) as f:
    data = ujson.load(f)

TRAINING_DATA = []

In [12]:
for d in data:
    tuple_1 = d['data']['response']
    tuple_2 = {}
    entities = []
    results = d['annotations'][0]['result']
    for x in results:
        entities.append((x['value']['start'],  x['value']['end'], x['value']['labels'][0]))
    tuple_2['entities'] = entities
    TRAINING_DATA.append((tuple_1, tuple_2))

In [13]:
from spacy.tokens import DocBin
import spacy
db = DocBin()
nlp = spacy.blank("en")

bad = 0

for text, annotations in TRAINING_DATA:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations['entities']:
        print(text, text[start:end], label)
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    try:
        doc.ents = ents
        db.add(doc)
    except:
        bad += 1
db.to_disk("./train.spacy")
print(bad)


kid's temp cappuchino kid's temp temperature
kid's temp cappuchino cappuchino drink
kid's temp mocha kid's temp temperature
kid's temp mocha mocha drink
kid's temp latte kid's temp temperature
kid's temp latte latte drink
grande chocolate almond shaken espresso grande size
grande chocolate almond shaken espresso shaken espresso drink
grande chocolate almond shaken espresso almond milk
grande chocolate almond shaken espresso chocolate syrup
tall lemonade with vanilla tall size
tall lemonade with vanilla lemonade drink
tall lemonade with vanilla vanilla syrup
espresso lemonade with vanilla espresso drink
espresso lemonade with vanilla lemonade syrup
espresso lemonade with vanilla vanilla syrup
white mocha with steamed soy white mocha  drink
white mocha with steamed soy steamed milk texture
white mocha with steamed soy soy milk
mocha big big size
mocha big mocha drink
vanilla steamer vanilla syrup
vanilla steamer steamer drink
big Italian with whole milk big size
big Italian with whole mi